In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [9]:
df = pd.read_csv('data/covid_19_data.csv',parse_dates=['Last Update'])
df.rename(columns={'ObservationDate':'Date', 'Country/Region':'Country'}, inplace=True)
df.tail()

,SNo,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
5885,5886,03/15/2020,Gibraltar,UK,2020-03-14 16:33:03,1.0,0.0,1.0
5886,5887,03/15/2020,NaN,Uzbekistan,2020-03-15 18:20:19,1.0,0.0,0.0
5887,5888,03/15/2020,Diamond Princess cruise ship,Australia,2020-03-14 02:33:04,0.0,0.0,0.0
5888,5889,03/15/2020,West Virginia,US,2020-03-10 02:33:04,0.0,0.0,0.0
5889,5890,03/15/2020,NaN,occupied Palestinian territory,2020-03-11 20:53:02,0.0,0.0,0.0


In [10]:
df2 = df.groupby(["Date", "Country"])[['Province/State','Date', 'Country', 'Confirmed', 'Deaths', 'Recovered']].sum().reset_index()
China_df = df2[df2['Country']=='Mainland China']
# get total confirmed growth rate
China_df['Confirmed Growth']= China_df['Confirmed'] / China_df['Confirmed'].shift()
# get new daily infections
China_df['Infections'] = China_df['Confirmed'].diff()
# find growth rate: infections(n)/infections(n-1)
China_df['Growth Rate']= China_df['Infections'] / China_df['Infections'].shift()

Italy_df = df2[df2['Country']=='Italy']
# get total confirmed growth rate
Italy_df['Confirmed Growth']= Italy_df['Confirmed'] / Italy_df['Confirmed'].shift()
# get new daily infections
Italy_df['Infections'] = Italy_df['Confirmed'].diff()
# find growth rate: infections(n)/infections(n-1)
Italy_df['Growth Rate']= Italy_df['Infections'] / Italy_df['Infections'].shift()


e:\python35\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

e:\python35\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

e:\python35\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

e:\python35\lib\

In [11]:
# clean data
China_df = China_df.fillna(0)
China_df = China_df.replace([np.inf, -np.inf], 1)
# get rolling average of Confirmed growth rate -- helps to smoothe data from outlier days
China_df['Conf Growth Roll Avg. 7']= China_df['Confirmed Growth'].rolling(7, win_type='triang').mean()
# get rolling average of growth rate -- helps to smoothe data from outlier days
China_df['Roll. Avg. 5']= China_df['Growth Rate'].rolling(5, win_type='triang').mean()

Italy_df = Italy_df.fillna(0)
Italy_df = Italy_df.replace([np.inf, -np.inf], 1)
# get rolling average of Confirmed growth rate -- helps to smoothe data from outlier days
Italy_df['Conf Growth Roll Avg. 7']= Italy_df['Confirmed Growth'].rolling(7, win_type='triang').mean()
# get rolling average of growth rate -- helps to smoothe data from outlier days
Italy_df['Roll. Avg. 5']= Italy_df['Growth Rate'].rolling(5, win_type='triang').mean()

In [12]:
def get_double_rate(df, column):
    double_day_rate = []
    for i in range(len(df[column])):
        for j in range(len(df[column])):
            if df[column].iloc[j] >= 2*df[column].iloc[i]:
                double_day_rate.append(j-i)
                break
        else:
            if j-i >= double_day_rate[-1]:
                double_day_rate.append(double_day_rate[-1]+1)
            else:
                double_day_rate.append(np.NaN)
    return double_day_rate

China_df['Days to Double'] = get_double_rate(China_df, 'Confirmed')
Italy_df['Days to Double'] = get_double_rate(Italy_df, 'Confirmed')

In looking at 'Days to Double', we see that China's aggressive quarantine policies cut the number of days it took the 
number of cases to double from 2 to 10 was 12, which looks to be much quicker than what Italy will show below.


In [13]:
China_df

,Date,Country,Confirmed,Deaths,Recovered,Confirmed Growth,Infections,Growth Rate,Conf Growth Roll Avg. 7,Roll. Avg. 5,Days to Double
3,01/22/2020,Mainland China,547.0,17.0,28.0,0.000000,0.0,0.000000,NaN,NaN,3.0
14,01/23/2020,Mainland China,639.0,18.0,30.0,1.168190,92.0,0.000000,NaN,NaN,2.0
28,01/24/2020,Mainland China,916.0,26.0,36.0,1.433490,277.0,3.010870,NaN,NaN,2.0
40,01/25/2020,Mainland China,1399.0,42.0,39.0,1.527293,483.0,1.743682,NaN,NaN,2.0
55,01/26/2020,Mainland China,2062.0,56.0,49.0,1.473910,663.0,1.372671,NaN,1.543627,2.0
72,01/27/2020,Mainland China,2863.0,82.0,58.0,1.388458,801.0,1.208145,NaN,1.689586,2.0
90,01/28/2020,Mainland China,5494.0,131.0,101.0,1.918966,2631.0,3.284644,1.366477,1.813020,4.0
109,01/29/2020,Mainland China,6070.0,133.0,120.0,1.104842,576.0,0.218928,1.476302,1.655742,4.0
130,01/30/2020,Mainland China,8124.0,171.0,135.0,1.338386,2054.0,3.565972,1.485538,1.960747,3.0
153,01/31/2020,Mainland China,9783.0,213.0,214.0,1.204210,1659.0,0.807692,1.469491,1.819317,3.0


In [16]:
Italy_df

,Date,Country,Confirmed,Deaths,Recovered,Confirmed Growth,Infections,Growth Rate,Conf Growth Roll Avg. 7,Roll. Avg. 5,Days to Double
150,01/31/2020,Italy,2.0,0.0,0.0,0.000000,0.0,0.000000,NaN,NaN,21.0
176,02/01/2020,Italy,2.0,0.0,0.0,1.000000,0.0,0.000000,NaN,NaN,20.0
203,02/02/2020,Italy,2.0,0.0,0.0,1.000000,0.0,0.000000,NaN,NaN,19.0
230,02/03/2020,Italy,2.0,0.0,0.0,1.000000,0.0,0.000000,NaN,NaN,18.0
258,02/04/2020,Italy,2.0,0.0,0.0,1.000000,0.0,0.000000,NaN,0.000000,17.0
286,02/05/2020,Italy,2.0,0.0,0.0,1.000000,0.0,0.000000,NaN,0.000000,16.0
314,02/06/2020,Italy,2.0,0.0,0.0,1.000000,0.0,0.000000,0.937500,0.000000,15.0
342,02/07/2020,Italy,3.0,0.0,0.0,1.500000,1.0,1.000000,1.031250,0.111111,14.0
371,02/08/2020,Italy,3.0,0.0,0.0,1.000000,0.0,0.000000,1.062500,0.222222,13.0
400,02/09/2020,Italy,3.0,0.0,0.0,1.000000,0.0,0.000000,1.093750,0.333333,12.0


In [28]:
# Total confirmed cases in China over time
# Quarantine measure were enacted in late January, which slowed growth, except for a sharp uptick around 2/13
# 2/13 uptick represents clinical diagnosis being treated on equal footing with lab testing

fig = px.line(China_df, x='Date', y='Confirmed')
fig.show()

In [29]:
# Total confirmed cases in Italy over time
fig = px.line(Italy_df, x='Date', y='Confirmed')
fig.show()

In [33]:
# Growth rate rolling avg. over previous 5 days -- Anything above 1 is exponential
fig = px.line(China_df[-18:], x='Date', y='Roll. Avg. 5')
fig.show()

In [34]:
# Growth rate rolling avg. over previous 5 days -- Anything above 1 is exponential
fig = px.line(Italy_df[-14:], x='Date', y='Roll. Avg. 5')
fig.show()

Italy Rolling Avg of new confirmations has dropped below one, signifying a slowing of the virus due to lockdown efforts.

In [58]:
list(China_df['Conf Growth Roll Avg. 7'].iloc[-43:-13])

[1.3333065376764426,
 1.2865422144485952,
 1.2511838934872888,
 1.2564241885343765,
 1.2255168347255125,
 1.1997351115032031,
 1.1663670954710765,
 1.1317727622226563,
 1.1117267858005517,
 1.0918711884215337,
 1.0752780826511237,
 1.0794496347005742,
 1.0896344300164935,
 1.102989994571699,
 1.1221737839190635,
 1.1058586010066274,
 1.0815323418081313,
 1.0566599737880182,
 1.0288530494654606,
 1.0191965707500845,
 1.0144228333182108,
 1.010913336006789,
 1.008737515056192,
 1.007886533569267,
 1.006492987384166,
 1.0057632125927451,
 1.0053251150551301,
 1.0047967095782733,
 1.0052706648682108,
 1.0052208784453278]

In [65]:
list(Italy_df['Conf Growth Roll Avg. 7'].iloc[-24:])

[1.3541666666666667,
 1.8395833333333336,
 2.41875,
 3.027755376344086,
 2.9538095154247075,
 2.6427906731388697,
 2.172141588027425,
 1.664047907988622,
 1.4762485943686396,
 1.4002060590844136,
 1.374792073205829,
 1.3490561579728846,
 1.329581824410518,
 1.28837212074931,
 1.262399727417197,
 1.2468615064244384,
 1.234739394971382,
 1.2380884277338993,
 1.2355056531112592,
 1.2278491673839116,
 1.2031544382074681,
 1.1896305749072589,
 1.1877715443983825,
 1.1848410901079534]